# Setup

On google colab, you have to restart runtime after running the following line

In [1]:
!pip install omegaconf

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
#"/content/drive/My Drive/NN-kNN/"
folder_name = "/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/"
import sys
sys.path.insert(0,folder_name)

Mounted at /content/drive/


In [3]:
##This is added because my Rdata uses Cdata for the covid data set.
##Rdata use Cdata function to load the data set, then convert it to regression problem
import os
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset')


In [4]:
# folder_name = os.getcwd()

In [5]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf

from dataset import cls_small_data as Cdata
import model.cls_model as Cmodel
from dataset import cls_medium_data

from dataset import reg_data as Rdata
import model.reg_model as Rmodel

In [6]:
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
torch.cuda.is_available()

True

# NCA and LMNN setup

In [9]:
pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.5 MB/s eta 0:00:00


In [10]:
import metric_learn
from metric_learn import LMNN,NCA

# Data Sets

Supported small dataset for classification:  
'zebra',
'zebra_special',
'bal',
'digits',
'iris',
'wine',
'breast_cancer',

for regression:
'califonia_housing',
'abalone',
'diabets',
'body_fat',
'ziweifaces'


Newly added data sets for mental health (psychology):

Classification:
'psych_depression_physical_symptons',
'covid_anxious',
'covid_depressed'


In [41]:
dataset_name = 'covid_physical'
cfg = conf_file['dataset'][dataset_name]
#TODO need to add other covid data sets here.
if dataset_name in ['covid_anxious','covid_depressed','covid_physical','covid_lonely','covid_hopeless',
                    'psych_depression_physical_symptons',
                    'zebra','zebra_special','bal','digits','iris','wine','breast_cancer']:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = Cdata.Cls_small_data(dataset_name)
elif dataset_name in []:
    criterion = torch.nn.CrossEntropyLoss()
    Xs, ys = cls_medium_data.Cls_medium_data(dataset_name)
else:
    criterion = torch.nn.MSELoss()
    Xs, ys = Rdata.Reg_data(dataset_name)

Columns in the dataset: Index(['SU_ID', 'P_PANEL', 'NATIONAL_WEIGHT', 'REGION_WEIGHT',
       'NATIONAL_WEIGHT_POP', 'REGION_WEIGHT_POP', 'NAT_WGT_COMB_POP',
       'REG_WGT_COMB_POP', 'P_GEO', 'SOC1',
       ...
       'REGION9', 'P_DENSE', 'MODE', 'LANGUAGE', 'MAIL50', 'RACE1_BANNER',
       'RACE2_BANNER', 'INC_BANNER', 'AGE_BANNER', 'HH_BANNER'],
      dtype='object', length=177)
   SOC1  SOC2A  SOC2B  SOC3A  SOC3B  SOC4A  SOC4B  PHYS8  PHYS1A  PHYS1B  \
0   3.0    4.0    4.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
1   3.0    3.0    3.0    1.0    3.0    2.0    2.0    2.0     2.0     2.0   
2   3.0    3.0    3.0    1.0    1.0    2.0    1.0    2.0     2.0     1.0   
3   3.0    2.0    2.0    1.0    1.0    2.0    1.0    3.0     2.0     2.0   
4   3.0    1.0    1.0    1.0    1.0    2.0    2.0    3.0     1.0     1.0   
5   2.0    3.0    3.0    1.0    1.0    1.0    1.0    2.0     2.0     2.0   
6   2.0    4.0    3.0    1.0    1.0    2.0    2.0    NaN     1.0     2.0   
7   3

In [42]:
# This section is used to reload the imported module.
# For example, if you made any changes in the model.cls_model, you should run importlib.reload(Cmodel) as long as you set import model.cls_model as Cmodel.
import importlib
importlib.reload(Rdata)
importlib.reload(Cdata)

<module 'dataset.cls_small_data' from '/content/drive/Othercomputers/My MacBook Pro/GitHub/NN-kNN/dataset/cls_small_data.py'>

In [43]:
#for reloading config file, in case you modified it for experimenting
conf_file = OmegaConf.load(os.path.join(folder_name, 'config.yaml'))
cfg = conf_file['dataset'][dataset_name]

In [44]:
cfg

{'ca_weight_sharing': True, 'top_case_enabled': False, 'training_epochs': 1000, 'learning_rate': 0.01, 'batch_size': 16, 'top_k': 5, 'class_weight_sharing': True, 'patience': 40, 'discount': 40}

# Classification with NNKNN

In [45]:
# prompt: get the unique y values and their counts

unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")
print(f"Xs.size(): {Xs.size()}")


Unique values: [0 1 2 3]
Counts: [115 115 115 115]
Xs.size(): torch.Size([460, 157])


In [46]:
def train_cls(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)

  # Train model
  # X_train_trimmed = X_train[:, :X_train.shape[1]//10]
  # y_train_trimmed = y_train[:X_train.shape[1]//10]
  model = Cmodel.NN_k_NN(X_train,
              y_train,
              cfg.ca_weight_sharing,
              cfg.top_case_enabled,
              cfg.top_k,
              cfg.discount,
              device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    epoch_msg = True

    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, output, predicted_class = model(X_train_batch)
      loss = criterion(output, y_train_batch)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

        epoch_msg = False
      # print("evaluating")
    model.eval()
    with torch.no_grad():
      _, _, output, predicted_class = model(X_test)

      # Calculate accuracy
      accuracy_temp = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)

    elif accuracy_temp > best_accuracy:
      #memorize best model
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0

    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  return best_accuracy, model

In [47]:
def load_model_cls(X_train,y_train,cfg):
  # Define the model architecture
  model = Cmodel.NN_k_NN(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model

In [48]:
accuracies = []
knn_accuracies = []
lmnn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/classifier_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle=True, random_state = None)
enable_lmnn = True

for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  if(enable_lmnn):
    # https://contrib.scikit-learn.org/metric-learn/supervised.html#lmnn
    lmnn = LMNN(n_neighbors=5, learn_rate=1e-6)
    ##TODO, change here if you need to use a different one
    # lmnn = metric_learn.MLKR()
    # lmnn = metric_learn.NCA(max_iter=1000)
    lmnn.fit(X_train,y_train)
    knn = KNeighborsClassifier(n_neighbors=5,metric=lmnn.get_metric())
    knn.fit(X_train,y_train)
    # klmnn_accuracies.append( accuracy_score(knn.predict(X_test), y_test))
    lmnn_acc = accuracy_score(knn.predict(X_test), y_test)
    lmnn_accuracies.append(lmnn_acc)
    print("lmnn_acc: ",lmnn_acc)
    # continue

  knn =  KNeighborsClassifier(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_acc  = accuracy_score(knn.predict(X_test), y_test)
  knn_accuracies.append(knn_acc)
  print("knn_acc: ", knn_acc)

  best_accuracy, model = train_cls(X_train,y_train, X_test, y_test, cfg)
  accuracies.append(best_accuracy)
  print("nnknn acc: ", best_accuracy)


print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")


lmnn_acc:  0.41304347826086957
knn_acc:  0.30434782608695654
Epoch [2/1000], Loss: 1.8669
Epoch [4/1000], Loss: 1.8907
Epoch [6/1000], Loss: 1.6683
Epoch [8/1000], Loss: 1.6024
Epoch [10/1000], Loss: 1.4150
Epoch [12/1000], Loss: 1.4198
Epoch [14/1000], Loss: 1.3935
Epoch [16/1000], Loss: 1.2275
Epoch [18/1000], Loss: 1.1313
Epoch [20/1000], Loss: 1.2395
Epoch [22/1000], Loss: 1.0801
Epoch [24/1000], Loss: 1.2571
Epoch [26/1000], Loss: 1.1764
Epoch [28/1000], Loss: 1.0724
Epoch [30/1000], Loss: 1.3278
Epoch [32/1000], Loss: 1.5138
Epoch [34/1000], Loss: 1.0134
Epoch [36/1000], Loss: 1.1752
Epoch [38/1000], Loss: 1.1704
Epoch [40/1000], Loss: 1.1890
Epoch [42/1000], Loss: 1.0524
Epoch [44/1000], Loss: 1.0644
Epoch [46/1000], Loss: 0.9777
Epoch [48/1000], Loss: 1.1051
patience exceeded, loading best model
nnknn acc:  0.43478260869565216
lmnn_acc:  0.2608695652173913
knn_acc:  0.2826086956521739
Epoch [2/1000], Loss: 1.9440
Epoch [4/1000], Loss: 2.2699
Epoch [6/1000], Loss: 1.8037
Epoch [

In [49]:
print(f"Average accuracy:{np.mean(accuracies):.3f}")
print(f"KNN accuracy:{np.mean(knn_accuracies):.3f}")
print(f"LMNN/NCA accuracy:{np.mean(lmnn_accuracies):.3f}")

Average accuracy:0.404
KNN accuracy:0.293
LMNN/NCA accuracy:0.326


# Regression with NNKNN

In [30]:
unique_values, counts = np.unique(ys, return_counts=True)
print(f"Unique values: {unique_values}")
print(f"Counts: {counts}")


Unique values: [-1.3401817  -0.44672722  0.44672722  1.3401817 ]
Counts: [115 115 115 115]


In [31]:
def train_reg(X_train,y_train, X_test, y_test, cfg:DictConfig):
  X_train = X_train.to(device)
  y_train = y_train.to(device)
  X_test = X_test.to(device)

  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=cfg.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=cfg.batch_size, shuffle=False)


    # Train model
  model = Rmodel.NN_k_NN_regression(X_train,
                                    y_train,
                                    cfg.ca_weight_sharing,
                                    cfg.top_case_enabled,
                                    cfg.top_k,
                                    cfg.discount,
                                    cfg.class_weight_sharing,
                                    device=device)

  optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate) #, weight_decay=1e-5)

  patience_counter = 0
  for epoch in range(cfg.training_epochs):
    # break # no training
    epoch_msg = True
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      _, _, _, predicted_number = model(X_train_batch)
      # break
      loss = criterion(predicted_number.squeeze(), y_train_batch)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if epoch_msg and (epoch + 1) % 2 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{cfg.training_epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
      predicted_numbers = []
      for X_test_batch, _ in test_loader:
        X_test_batch = X_test_batch.to(device)
        _, _, _, predicted_number = model(X_test_batch)
        predicted_numbers.extend(predicted_number.squeeze().cpu().detach())

      predicted_numbers = torch.Tensor(predicted_numbers)
      accuracy_temp = criterion(y_test, predicted_numbers)

    if epoch == 0:
      best_accuracy = accuracy_temp
      torch.save(model.state_dict(), cfg.PATH)
    elif accuracy_temp < best_accuracy:
      torch.save(model.state_dict(), cfg.PATH)
      best_accuracy = accuracy_temp
      patience_counter = 0
    elif patience_counter > cfg.patience:
      model.eval()
      print("patience exceeded, loading best model")
      break
    else:
      patience_counter += 1

  _, case_activations, topk_avg, predicted_number = model(X_test)

  top_case_indices = torch.topk(case_activations, 5, dim=1)[1].cpu()

  accuracy = criterion(y_test, predicted_number.squeeze().cpu())
  y_train = y_train.cpu()
  top_k_average_accuracy = mean_squared_error(torch.mean(y_train[top_case_indices], dim=1), y_test)
  # top_k_average_accuracy = mean_squared_error(topk_avg.cpu(), y_test)
  return best_accuracy, accuracy, top_k_average_accuracy, model

In [32]:
# prompt: load_model_reg()

def load_model_reg(X_train,y_train,cfg):
  # Define the model architecture
  model = Rmodel.NN_k_NN_regression(
      X_train,
      y_train,
      cfg.ca_weight_sharing,
      cfg.top_case_enabled,
      cfg.top_k,
      cfg.discount,
      cfg.class_weight_sharing,
      device=device
  )
  # Load the state dictionary
  model.load_state_dict(torch.load(cfg.path))
  model.to(device)
  model.eval()
  return model


In [33]:
best_accuracies = []
accuracies = []
top_k_average_accuracies = []
knn_accuracies = []
PATH = os.path.join(folder_name, f'checkpoints/regression_{dataset_name}.h5')
cfg.PATH = PATH
k_fold = KFold(n_splits=10, shuffle = True,random_state = None)


for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]

  knn = KNeighborsRegressor(n_neighbors=cfg.top_k)
  knn.fit(X_train, y_train)
  knn_accuracies.append(mean_squared_error(knn.predict(X_test), y_test))

  best_accuracy, accuracy, top_k_average_accuracy, model= train_reg(X_train, y_train, X_test, y_test, cfg)
  best_accuracies.append(best_accuracy)
  accuracies.append(accuracy)
  top_k_average_accuracies.append(top_k_average_accuracy)
  # break

print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Epoch [2/2000], Loss: 0.9824
Epoch [4/2000], Loss: 0.9979
Epoch [6/2000], Loss: 1.1160
Epoch [8/2000], Loss: 1.2820
Epoch [10/2000], Loss: 0.9681
Epoch [12/2000], Loss: 1.0968
Epoch [14/2000], Loss: 1.0549
Epoch [16/2000], Loss: 1.0445
Epoch [18/2000], Loss: 0.9969
Epoch [20/2000], Loss: 1.0323
Epoch [22/2000], Loss: 0.8884
Epoch [24/2000], Loss: 1.0314
Epoch [26/2000], Loss: 0.9428
Epoch [28/2000], Loss: 1.1208
Epoch [30/2000], Loss: 0.9594
Epoch [32/2000], Loss: 1.0688
Epoch [34/2000], Loss: 1.0028
Epoch [36/2000], Loss: 0.8363
Epoch [38/2000], Loss: 0.8800
Epoch [40/2000], Loss: 0.6802
Epoch [42/2000], Loss: 0.9955
patience exceeded, loading best model
Epoch [2/2000], Loss: 0.8782
Epoch [4/2000], Loss: 0.9787
Epoch [6/2000], Loss: 1.0730
Epoch [8/2000], Loss: 1.0274
Epoch [10/2000], Loss: 1.0489
Epoch [12/2000], Loss: 0.9963
Epoch [14/2000], Loss: 1.2338
Epoch [16/2000], Loss: 1.3437
Epoch [18/2000], Loss: 1.0548
Epoch [20/2000], Loss: 1.1276
Epoch [22/2000], Loss: 1.0233
Epoch [24/

In [34]:
print("Average accuracy:", np.mean([acc.detach().numpy() for acc in accuracies]))
print("Average top_k_average_accuracies", np.mean(top_k_average_accuracies))
print("KNN accuracy:", np.mean(knn_accuracies))

Average accuracy: 0.99592906
Average top_k_average_accuracies 1.0465546
KNN accuracy: 1.0793874


# Results Interpretation

In [35]:
def print_model_features(input_model):
  for n, p in model.named_parameters():
    print(n)
    print(p.data)

In [36]:
print_model_features(model)

fa_layer.f1weight
tensor([1.0646, 0.9778, 0.9870, 0.9357, 1.0788, 1.0091, 0.9982, 1.1459, 1.0635,
        1.0747, 1.1307, 1.0958, 0.9828, 1.1748, 1.0513, 1.1137, 1.0952, 1.1366,
        1.0851, 1.1076, 1.0556, 1.0433, 1.1043, 1.1563, 1.0348, 1.0374, 0.9015,
        1.1102, 1.1140, 0.9573, 1.0733, 1.1348, 1.1363, 1.0109, 1.0304, 1.1578,
        1.1615, 1.0784, 1.1400, 1.1086, 0.9542, 1.0675, 1.1521, 1.0411, 1.0000,
        1.0167, 1.0000, 1.1669, 1.1556, 1.1200, 1.0148, 1.1048, 0.9978, 0.9632,
        1.1496, 1.0648, 0.9445, 1.0137, 1.0515, 1.1217, 1.0101, 1.1168, 0.9894,
        1.1180, 1.1682, 0.9748, 1.0030, 1.1053, 1.0726, 1.1410, 1.1338, 1.0042,
        0.9760, 1.0885, 0.9990, 0.9267, 0.9480, 0.9499, 1.0616, 1.0000, 1.0046,
        1.0000, 1.0028, 0.9786, 1.1039, 1.0028, 1.0764, 1.1090, 1.0560, 1.0780,
        1.0275, 1.0775, 1.0058, 1.0164, 0.9759, 0.9627, 1.0177, 0.9365, 1.0740,
        0.9542, 1.1459, 1.1218, 1.0299, 1.0387, 1.0700, 1.1087, 1.0000, 0.9476,
        1.0000, 1.0256

In [37]:
# for regression only. for classification is different
#feature_activations, case_activations, predicted_number
model.eval()
feature_activations, case_activations, output, predicted_class = model(X_test.to(device))

In [38]:
predicted_class

tensor([ 4.1465e-02,  5.3894e-02, -2.0531e-02, -8.6567e-02, -8.3351e-02,
         1.7569e-09, -8.1551e-02,  4.3990e-02, -4.9926e-03,  2.4224e-01,
        -2.6790e-02,  4.4673e-01, -8.4768e-02,  1.2664e-01,  8.0771e-02,
         6.5137e-02, -6.7750e-02, -1.1325e-02, -8.2301e-02, -7.8184e-02,
         1.0249e-01, -5.7211e-02,  7.6599e-02, -1.1510e-01, -2.7233e-02,
        -7.1776e-02, -8.7807e-02, -7.6512e-02,  1.8108e-01, -1.5044e-01,
        -7.1940e-01, -3.0080e-01,  6.7821e-02, -8.6528e-02, -6.2181e-02,
         1.3165e-01, -2.3978e-01, -2.1318e-02,  3.3745e-02,  2.5853e-01,
        -8.5523e-02,  1.6500e-01, -2.5566e-02, -1.1269e-02, -3.5682e-02,
        -6.0301e-03], device='cuda:0', grad_fn=<MvBackward0>)

In [39]:
y_test

tensor([-0.4467,  0.4467, -1.3402, -1.3402,  0.4467,  0.4467, -0.4467, -1.3402,
        -1.3402, -1.3402, -0.4467, -1.3402,  0.4467,  1.3402,  0.4467, -0.4467,
        -0.4467,  1.3402,  1.3402, -0.4467,  0.4467, -1.3402,  1.3402,  1.3402,
         1.3402, -1.3402,  0.4467,  0.4467, -0.4467, -1.3402,  1.3402,  0.4467,
        -0.4467, -0.4467,  1.3402,  0.4467,  1.3402,  1.3402, -1.3402, -0.4467,
        -1.3402,  0.4467,  0.4467, -0.4467,  0.4467,  0.4467])

In [40]:
# prompt: accuracy comparing predicted_class and y_test

accuracy = accuracy_score(y_test.numpy(), predicted_class.cpu().numpy())
print("Accuracy:", accuracy)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
#inspecting the case activations
top_case_indices = torch.topk(case_activations, 5, dim=1)[1]

In [ ]:
X_test[0]

In [ ]:
y_test[0]

In [ ]:
X_train[top_case_indices[0][0]]

In [ ]:
y_train[top_case_indices[0][0]]

By comparing the following two blocks' outputs, you can see we are retrieving a good neighbor.

In [ ]:
#sum abs of X_test[0] and the top activated case
sum(abs(X_test[0] - X_train[top_case_indices[0][0]]))

In [ ]:
# prompt: average sum abs of X_test[0] and X_train data
print(np.mean([sum(abs(X_test[0] - X_train[i])) for i in range(len(X_train))]))

TODO:: A better way is to show the distribution of ``X_test[0] - X_train[i]``

In [ ]:
y_train[top_case_indices[0]]

In [ ]:
knn.predict(X_test)[0]

In [ ]:
indices = knn.kneighbors(X_test)[1][0]

In [ ]:
y_train[indices]

# Sanity Check


## Classification Neural Network

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 1024
num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network architecture for classification
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, num_classes)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x)


In [ ]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  # Initialize the model, loss function, and optimizer
  model = NeuralNet(input_size, hidden_size, num_classes)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Training loop
  for epoch in range(epochs):
    epoch_msg = True
    training_total_acc = 0.0
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)

      # Backward and optimize
      _, predicted = torch.max(outputs, 1)
      training_total_acc += torch.sum(predicted == y_train_batch).item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      training_total_loss += loss.item()
      # if (i + 1) % 5 == 0
    if epoch == 0 or (epoch + 1) % 100 == 0:
      print(f"Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f} Acc: {training_total_acc/num_of_batches:.2f}")
    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      _, predicted = torch.max(outputs, 1)
      accuracy = torch.sum(predicted == y_test).item() / len(y_test)
      print(f'Accuracy on the test set: {accuracy * 100:.2f}%')
      if best_accuracy is None or accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epoch}], Test Loss: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best acc achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
  # break
print("Average accuracy:", np.mean(best_accuracies))

Epoch: 1, Training Loss: 1.13 Acc: 4.00
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 41.67%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 50.00%
Accuracy on the test set: 50.00%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Accuracy on the test set: 58.33%
Acc

## Regression Neural Network

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 100
# num_classes = torch.unique(ys).shape[0]
learning_rate = 1e-5
batch_size = 16
epochs = 2000

In [ ]:
# prompt: a standard neural network with 3 fully connected layers for regression

import torch
import torch.nn as nn
import torch.nn.functional as F

class RegressionNet(nn.Module):
    def __init__(self, input_size):
        super(RegressionNet, self).__init__()
        self.nn = nn.Sequential(
            nn.Linear(input_size, hidden_size ),
            nn.LeakyReLU(),
            nn.Linear(hidden_size , hidden_size // 2),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            # nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size // 4, 1)
            )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)

    def forward(self, x):
        return self.nn(x).squeeze()

In [ ]:
best_accuracies = []
for train_index, test_index in k_fold.split(Xs):
  # Get training and testing data
  X_train, X_test = Xs[train_index], Xs[test_index]
  y_train, y_test = ys[train_index], ys[test_index]
  train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
  patience_counter = 0
  best_model = None
  best_accuracy = None
  model = RegressionNet(Xs.shape[1])
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  for epoch in range(epochs):
    epoch_msg = True
    training_total_loss = 0.0
    num_of_batches = len(train_loader)
    for X_train_batch, y_train_batch in train_loader:
      model.train()
      # Forward pass
      outputs = model(X_train_batch)
      loss = criterion(outputs, y_train_batch)
      training_total_loss += loss.item()
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch == 0 or (epoch + 1) % 3 == 0:
      print(f'Epoch: {epoch + 1}, Training Loss: {training_total_loss/num_of_batches:.2f}')

    # Testing the model
    model.eval()
    with torch.no_grad():
      outputs = model(X_test)
      loss = criterion(outputs, y_test)
      if best_accuracy is None or loss.item() < best_accuracy:
        best_accuracy = loss.item()
        best_model = model
        patience_counter = 0
      else:
        patience_counter += 1
      if epoch_msg and (epoch + 1) % 100 == 0:
        epoch_msg = False
        print(f'Epoch [{epoch + 1}/{epochs}], Test Loss: {loss.item()}')
        # print(f'Loss on the test set: {loss.item()}')
    if patience_counter >= cfg.patience:
      print("Best loss achieved: ", best_accuracy)
      break
  best_accuracies.append(best_accuracy)
print("Average accuracy:", np.mean(best_accuracies))
